In [1]:
import os
import numpy as np
import pandas as pd
import time
import random
from tqdm import tqdm
from sklearn import preprocessing
import math
import matplotlib
import matplotlib.pyplot as plt
import csv

In [2]:
grid_size = 10000 # pixels, bigger than 0 and int
total_num_nodes = 100 # bigger than 0 and int
num_activate_nodes = 2 # must be even, int
num_inactivate_nodes = total_num_nodes - num_activate_nodes
nodes_generate_model = 'gauss' # uniform, gauss, random
paired_nodes = np.array([[8,12],[33,44]])
aware_range = 1
ER_cone_angle = 30 # degree not radian
ER_radius = 1
ER_cone_length = 1

In [3]:
def grid_generation(size):
    grid_array = np.zeros([size,size])
    return grid_array

In [4]:
def all_nodes_position(nums, maxsize, model_flag):
    if model_flag == 'random':
        return np.random.randint(maxsize, size=(nums, 2))
    if model_flag == 'uniform':
        return (maxsize * (np.random.random_sample((nums, 2)))).astype(int)
    if model_flag == 'gauss':
        return np.random.normal(maxsize/2, maxsize/7, size=(nums, 2)).astype(int) # mu set to /7 to make sure all int bigger than 0
    else:
        return "error: invalid nodes generate model!"

In [5]:
def sort_nodes(nodes, nums):
    rank = [*range(1, nums+1, 1)]
    zero = [0] * nums
    new_nodes = np.insert(nodes, 2, zero, axis=1)
    new_nodes = np.insert(new_nodes, 3, rank, axis=1)
    new_nodes = np.insert(new_nodes, 4, zero, axis=1)
    new_nodes = np.insert(new_nodes, 4, zero, axis=1)
    new_nodes = np.insert(new_nodes, 4, zero, axis=1)
    return new_nodes

In [6]:
def nodes_activate(node, order, status, up, down, des):
    node[order][2] = status # initial Rx
    node[order][4] = up
    node[order][5] = down
    node[order][6] = des
    return node

In [7]:
def ER_area(ER_cone_angle, ER_radius, ER_cone_length):
    return

In [8]:
def upstream_node_info(node, order):
    return 

In [9]:
def downstream_node_info(node, order):
    return 

In [ ]:
def draw_network(position,\
                 link_table,\
                 link_flow_information=[],\
                 cancave_size=[4000,4000],\
                 number_of_nodes = 25,\
                 heat_map_ground_truth = np.zeros((32,32)),\
                 label_number = 5):
    
    base_line_traffic = 500
fig = plt.figure() 
ax = fig.add_subplot(111) 
    #---------- plot background traffics ------------
    #optional_background_color = ['darkred','orangered','orange','navajowhite','white']
optional_background_color = ['whitesmoke','navajowhite','orange','orangered','darkred']
total_color_number = len(optional_background_color)
    x_ratio = float(cancave_size[0]/heat_map_ground_truth.shape[0])
    y_ratio = float(cancave_size[1]/heat_map_ground_truth.shape[1])
    width = x_ratio
    height = y_ratio
    #print('check width height ',width, height)
    for row_number in range(heat_map_ground_truth.shape[0]):
        for col_number in range(heat_map_ground_truth.shape[1]):
            back_ground_vlaue = heat_map_ground_truth[row_number][col_number]
            convert_to_color_index = back_ground_vlaue/(label_number-1)*(total_color_number-1)
            #print(back_ground_vlaue,convert_to_color_index)
            use_color = optional_background_color[int(convert_to_color_index)]
            x_corner = row_number*x_ratio
            y_corner = col_number*y_ratio
            rect = matplotlib.patches.Rectangle((x_corner,y_corner), width ,height, facecolor=use_color)
            ax.add_patch(rect)
            #print(x_corner,y_corner,use_color)
            #PIL.ImageDraw.Draw.rectangle([(x_corner),(y_corner),(x_corner+width),(y_corner+height)], fill=use_color, outline=None)
    #fig.savefig('test.png', dpi=90, bbox_inches='tight')
    plt.xlim(0, cancave_size[0])
    plt.ylim(0, cancave_size[1]) 
    #plt.show()
    #---------- plot nodes and traffics-------------
    text =[]
    for a in range(number_of_nodes):
        text.append(str(a))
    #print(text)
    for a in range(number_of_nodes):
        plt.annotate(text[a],(position[a][0]+20, position[a][1]+20))
        plt.scatter(position[a][0], position[a][1])
    for a in range(number_of_nodes):
        point1=[position[a][0],position[a][1]]
        for b in range(number_of_nodes):
            point2=[position[b][0],position[b][1]]
            if link_table[a][b] == 1:
                x_values = [point1[0], point2[0]]
                y_values = [point1[1], point2[1]]
                plt.plot(x_values, y_values, linestyle='dashed')
    if len(link_flow_information)>0:
        total_number_of_flows = link_flow_information[-1][0]
        color_options = ['b', 'g', 'r', 'c', 'm', 'y', 'k', 'w']
        total_color_options = len(color_options)
        for a in link_flow_information:
            flow_number = a[0]
            start_node = int(a[1])
            end_node = int(a[2])
            traffic_on_link = a[3]
            point1=[position[start_node][0],position[start_node][1]]
            point2=[position[end_node][0],position[end_node][1]]
            x_values = [point1[0], point2[0]]
            y_values = [point1[1], point2[1]]
            plt.plot(x_values, \
                     y_values,\
                     color = color_options[int(flow_number%total_color_options)], \
                     linewidth=traffic_on_link/base_line_traffic)
    plt.show()        

In [11]:
# ENV INITIALIZATION
grid_array = grid_generation(grid_size)
nodes_position = all_nodes_position(total_num_nodes, grid_size, nodes_generate_model)
# sorted and ready nodes
# [xaxis, yaxis, status, #, upstream, downstream]
# status 0 inactivate 1 Tx 2 Rx
nodes_position = sort_nodes(nodes_position, total_num_nodes)
# activate_nodes
for [i,j] in paired_nodes:
    nodes_position = nodes_activate(nodes_position, i-1, 1, 1, 4, j)
    nodes_position = nodes_activate(nodes_position, j-1, 2, 67, 99, i)
#     nodes_position = nodes_activate(nodes_position, i-1, 1, upstream_node_info(nodes_position, i-1), downstream_node_info(nodes_position, i-1), j)
#     nodes_position = nodes_activate(nodes_position, j-1, 2, upstream_node_info(nodes_position, j-1), downstream_node_info(nodes_position, j-1), i)
    
    
    
print(grid_array.shape)
print(nodes_position)



(10000, 10000)
[[5807 6939    0    1    0    0    0]
 [6494 5355    0    2    0    0    0]
 [6622 4426    0    3    0    0    0]
 [1734 7089    0    4    0    0    0]
 [4576 3224    0    5    0    0    0]
 [5061 7505    0    6    0    0    0]
 [4227 4524    0    7    0    0    0]
 [3749 5598    1    8    1    4   12]
 [3906 5716    0    9    0    0    0]
 [4991 3507    0   10    0    0    0]
 [4718 6018    0   11    0    0    0]
 [5321 4819    2   12   67   99    8]
 [4968 4892    0   13    0    0    0]
 [5836 3741    0   14    0    0    0]
 [6609 3042    0   15    0    0    0]
 [6963 4954    0   16    0    0    0]
 [4142 2546    0   17    0    0    0]
 [4650 2376    0   18    0    0    0]
 [4820 5087    0   19    0    0    0]
 [5956 5994    0   20    0    0    0]
 [6614 4729    0   21    0    0    0]
 [5016 5027    0   22    0    0    0]
 [5060 4624    0   23    0    0    0]
 [4403 5451    0   24    0    0    0]
 [3659 3933    0   25    0    0    0]
 [4943 5410    0   26    0    0    